In [8]:
import pandas as pd
import os
import shutil

# STOCKS DATA
df = pd.read_excel('tickersymbols.xls')	
df.head()

In [92]:
# Considering five indexes was proposed of which four are considered 
# since the nyseUS100 closely follows the nasdaq100,only nasdaq100 is considered to avoid any redundant analysis

# Getting all tickers as strings
nasdaq100 = list(df['nasdaq100'])
nasdaq100Tech = list(df['nasdaq100Tech'])
# not considering the missing values in nasdaq100Tech (30 values only as seen in the excel sheet)
nasdaq100Tech = nasdaq100Tech[:29] 
sp100 = list(df['sp100'])
# not considering the missing values in sp100 (100 values only as seen in the excel sheet)
sp100 = sp100[:99]
# removing the formatting letter at front
sp100 = [x[1:] for x in sp100]
nyseArcaTech100 = list(df['nyseArcaTech100'])
# not considering the missing values in nyseArcaTech100 (30 values only as seen in the excel sheet)
nyseArcaTech100 = nyseArcaTech100[:29]

In [95]:
# taking only those files that are in the dataset

files = os.listdir('stock_market_data/nasdaq/csv/')
for x in nasdaq100:
    if x+'.csv' in files:
        shutil.copy('stock_market_data/nasdaq/csv/'+x+'.csv', 'stock_market_data/nasdaq100')
        
for x in nasdaq100Tech:
    if x+'.csv' in files:
        #print(x)
        shutil.copy('stock_market_data/nasdaq/csv/'+x+'.csv', 'stock_market_data/nasdaq100Tech')

files = os.listdir('stock_market_data/sp500/csv/')
for x in sp100:
    if x+'.csv' in files:
        shutil.copy('stock_market_data/sp500/csv/'+x+'.csv', 'stock_market_data/sp100')

files = os.listdir('stock_market_data/nyse/csv/')
for x in nyseArcaTech100:
    if x+'.csv' in files:
        shutil.copy('stock_market_data/nyse/csv/'+x+'.csv', 'stock_market_data/nyseArcaTech100')
    

IBM
JNPR


In [96]:
files = os.listdir('stock_market_data/nasdaq100')
print(len(files))
files = os.listdir('stock_market_data/nasdaq100Tech')
print(len(files))
files = os.listdir('stock_market_data/sp100')
print(len(files))
files = os.listdir('stock_market_data/nyseArcaTech100')
print(len(files))


72
20
68
2


In [4]:
# Not every stock data is present in the dataset for every stock in the index
#    so that will be one drawback of this dataset

# Only two files for the nyseArcaTech Index were found
# Hence no point in considering this index
# Finally, only three indices are considered - nasdaq100, sp100, nasdaq100Tech

In [1]:

# NOTE: DO NOT RUN THIS SEGMENT IF YOU ALREADY CREATED THE COMBINED FILE IN THE DIRECTORY. IT WILL COMBINE THAT FILE ALSO.
# Combining them all into a single csv file
import os
import glob
import pandas as pd
# nasdaq100
os.chdir("stock_market_data/nasdaq100")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ]).reset_index()
#export to csv
combined_csv.to_csv( "nasdaq100.csv", index=False, encoding='utf-8-sig')

In [2]:

# NOTE: DO NOT RUN THIS SEGMENT IF YOU ALREADY CREATED THE COMBINED FILE IN THE DIRECTORY. IT WILL COMBINE THAT FILE ALSO.
# sp100
os.chdir("../sp100")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ]).reset_index()
#export to csv
combined_csv.to_csv( "sp100.csv", index=False, encoding='utf-8-sig')

In [5]:

# NOTE: DO NOT RUN THIS SEGMENT IF YOU ALREADY CREATED THE COMBINED FILE IN THE DIRECTORY. IT WILL COMBINE THAT FILE ALSO.
# nasdaq100Tech
os.chdir("../nasdaq100Tech")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ]).reset_index()
#export to csv
combined_csv.to_csv( "nasdaq100Tech.csv", index=False, encoding='utf-8-sig')

In [13]:

# Reading the data into spark for some basic EDA
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [98]:
# nasdaq100
df = sqlContext.read.csv('../nasdaq100/nasdaq100.csv',header=True)
print(df.rdd.count())
# Selecting the data within a time range
from datetime import datetime
def get_date_time_obj(date_time_str):
    date = datetime.strptime(date_time_str, '%d-%m-%Y')
    return date
min_date = get_date_time_obj('01-07-2018')
max_date = get_date_time_obj('01-07-2021')
# Use filterbyvalues to select those files in the range
filtered_rdd = df.rdd.filter(lambda x: get_date_time_obj(x[1])>=min_date and get_date_time_obj(x[1])<=max_date)
print(filtered_rdd.count())
# Further EDA
# Checking to see if data is present for all 1097 dates from 1st Jul 2018 to 1st Jul 2021
# counting by the number of entries for each date
value_count_for_each_date = filtered_rdd.map(lambda x: (x[1],1)) \
                    .reduceByKey(lambda a,b: a+b).collect()
print(len(value_count_for_each_date))
# only 756 values
# can be explained as this dataset is not updated daily. Hence, there won't be all 1097 values
# but can proceed with this as it has seven data points at regularly for every month and will contain the trend information
# a peak into the latest counts
value_count_for_each_date[-15:]
# checking for missing entries for any stock
counts = set([x[1] for x in value_count_for_each_date ])
print(counts)
# 72 is the only unique count value
# So 756 values present for all 72 stocks. No missing entries for any date!

516503
54432
756
{72}


In [99]:
# checking for any other missing values
df = sqlContext.createDataFrame(filtered_rdd)
print(df.na.drop().rdd.count())
# counts before and after removing missing values is the same
# Hence no missing values
# also cross checking for any missing values with pandas
df_pd = pd.read_csv('../nasdaq100/nasdaq100.csv')
df_pd.isna().sum()

54432


index             0
Date              0
Low               0
Open              0
Volume            0
High              0
Close             0
Adjusted Close    0
dtype: int64

In [100]:
# saving the file
os.chdir('../nasdaq100')
df.write.csv('../nasdaq100/nasdaq100_final')
!cat nasdaq100_final/part* > nasdaq100_final.csv

In [90]:
# sp100
df = sqlContext.read.csv('../sp100/sp100.csv',header=True)
print(df.rdd.count())
# Selecting the data within a time range
from datetime import datetime
def get_date_time_obj(date_time_str):
    date = datetime.strptime(date_time_str, '%d-%m-%Y')
    return date
min_date = get_date_time_obj('01-07-2018')
max_date = get_date_time_obj('01-07-2021')
# Use filterbyvalues to select those files in the range
filtered_rdd = df.rdd.filter(lambda x: get_date_time_obj(x[1])>=min_date and get_date_time_obj(x[1])<=max_date)
print(filtered_rdd.count())
# Further EDA
# Checking to see if data is present for all 1097 dates from 1st Jul 2018 to 1st Jul 2021
# counting by the number of entries for each date
value_count_for_each_date = filtered_rdd.map(lambda x: (x[1],1)) \
                    .reduceByKey(lambda a,b: a+b).collect()
print(len(value_count_for_each_date))
# similarly, only 756 values
# a peak into the latest counts
value_count_for_each_date[-15:]
# checking for missing entries for any stock
counts = set([x[1] for x in value_count_for_each_date ])
print(counts)
# 68 is the only unique count value
# So 756 values present for all 68 stocks. No missing entries for any date!

639414
51408
756
{68}


In [91]:
# checking for any other missing values
df = sqlContext.createDataFrame(filtered_rdd)
print(df.na.drop().rdd.count())
# counts before and after removing missing values is the same
# Hence no missing values
# also cross checking for any missing values with pandas
df_pd = pd.read_csv('../sp100/sp100.csv')
df_pd.isna().sum()

51408


index             0
Date              0
Low               0
Open              0
Volume            0
High              0
Close             0
Adjusted Close    0
dtype: int64

In [93]:
# saving the file
os.chdir('../sp100')
df.write.csv('../sp100/sp100_final')
!cat sp100_final/part* > sp100_final.csv

In [94]:
# nasdaq100Tech
df = sqlContext.read.csv('../nasdaq100Tech/nasdaq100Tech.csv',header=True)
print(df.rdd.count())
# Selecting the data within a time range
from datetime import datetime
def get_date_time_obj(date_time_str):
    date = datetime.strptime(date_time_str, '%d-%m-%Y')
    return date
min_date = get_date_time_obj('01-07-2018')
max_date = get_date_time_obj('01-07-2021')
# Use filterbyvalues to select those files in the range
filtered_rdd = df.rdd.filter(lambda x: get_date_time_obj(x[1])>=min_date and get_date_time_obj(x[1])<=max_date)
print(filtered_rdd.count())
# Further EDA
# Checking to see if data is present for all 1097 dates from 1st Jul 2018 to 1st Jul 2021
# counting by the number of entries for each date
value_count_for_each_date = filtered_rdd.map(lambda x: (x[1],1)) \
                    .reduceByKey(lambda a,b: a+b).collect()
print(len(value_count_for_each_date))
# similarly, only 756 values
# a peak into the latest counts
value_count_for_each_date[-15:]
# checking for missing entries for any stock
counts = set([x[1] for x in value_count_for_each_date ])
print(counts)
# 20 is the only unique count value
# So 756 values present for all 20 stocks. No missing entries for any date!

156839
15120
756
{20}


In [95]:
# checking for any other missing values
df = sqlContext.createDataFrame(filtered_rdd)
print(df.na.drop().rdd.count())
# counts before and after removing missing values is the same
# Hence no missing values
# also cross checking for any missing values with pandas
df_pd = pd.read_csv('../nasdaq100Tech/nasdaq100Tech.csv')
df_pd.isna().sum()

15120


index             0
Date              0
Low               0
Open              0
Volume            0
High              0
Close             0
Adjusted Close    0
dtype: int64

In [96]:
# saving the file
os.chdir('../nasdaq100Tech')
df.write.csv('../nasdaq100Tech/nasdaq100Tech_final')
!cat nasdaq100Tech_final/part* > nasdaq100Tech_final.csv